# 目录



| English Name of Puzzles | Chinese Translation | Finished? |
| :-----: | :-----: | :----: |
| A Standard Sudoku  | 标准数独 | ✅ |
| An Alphadoku | 25 by 25 字母独| ✅  |
| A Killer Sudoku | 杀手数独 | ✅  |
| A Small Killer Sudoku | 小杀手数独 | ❌ |
| A Consecutive Sudoku | 连续数独 | ✅ |
| Akari (AKA: light UP!) | 照明 | ✅ |
| Cryptarithmetic Puzzle | 破译密码 | ✅ |
| Norinori  | 海苔 | ✅ |
| A thermometer Sudoku | 温度计数独 | ❌ |
| Siltherlink | 环 | ❌ |
| Number Link | 数链 | ✅ |
| A Minesweeper | 静态扫雷 | ✅  |


# Reference

> https://github.com/dguzelkokar/Solving-Akari-using-Google-OR-Tools
> 
> src: https://www.kaggle.com/code/pintowar/modeling-a-sudoku-solver-with-or-tools
> 
> src: http://www.hakank.org/google_or_tools/
>
> [Puzzle Site](https://puzzlemadness.co.uk/slitherlink/large/2023/10/18)


# 解谜游戏：数独 (Sudoku)

标准数独规则：

- 每行、每列、每个九宫格内填入1 ～ 9不重复、不缺漏出现

---------

Standard Sudoku Rules:

- Fill 1~9 to every row / column and cage, making sure all of them appear once and have no repetition each row / col / cage


In [1]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

def main(grid, grid_size):

    model = cp.CpModel()
    n = grid_size
    x = {}
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i * grid_size  + j] != "0":
                x[i, j] = int(grid[i * grid_size + j]) 
                # 这里规定：初始已经给了的数字不变
            else:
                x[i, j] = model.NewIntVar(1, grid_size, 'x[{},{}]'.format(i,j) ) 
                # 这里规定：余下的数字从1到9
    
    # ======= 下面是约束 ==========
    # 行和列必须不重复
    for i in range(n):
        row = [x[i, j] for j in range(n)]
        model.AddAllDifferent(row)
        col = [x[j, i] for i in range(n)]
        model.AddAllDifferent(col)

    # 3 * 3 的宫内数字不能重复
    for i in range(2):
        for j in range(2):
            cell = [
                x[r, c]
                for r in range(i * 3, i * 3 + 3)
                for c in range(j * 3, j * 3 + 3)
            ]
            model.AddAllDifferent(cell)
    solver = cp.CpSolver() 
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(n):
            for j in range(n):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


if __name__ == "__main__":
    main("549001738367008001200073040000900005000705460135840070004000307780350006023080000", 9)

5 4 9 2 6 1 7 3 8 
3 6 7 4 9 8 5 2 1 
2 1 8 5 7 3 6 4 9 
4 7 6 9 3 2 8 1 5 
8 9 2 7 1 5 4 6 3 
1 3 5 8 4 6 9 7 2 
6 5 4 1 2 9 3 8 7 
7 8 1 3 5 4 2 9 6 
9 2 3 6 8 7 1 5 4 

NumConflicts: 0
NumBranches: 0
WallTime: 0.004422000000000001


# 解谜游戏：Alphabetoku

25 * 25 的大规模数独

----

English Version:

- Standard Sudoku Rules Applied.
- Size 5 * 5



In [ ]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

str2Int = {
    "A" : 1,
    "B" : 2,
    "C" : 3,
    "D" : 4,
    "E" : 5,
    "F" : 6,
    "G" : 7,
    "H" : 8,
    "I" : 9,
    "J": 10,
    "K": 11,
    "L": 12,
    "M": 13,
    "N": 14,
    "O": 15,
    "P": 16,
    "Q": 17,
    "R": 18,
    "S": 19,
    "T": 20,
    "U": 21,
    "V": 22,
    "W": 23,
    "X": 24,
    "Y": 25,
}

def main_25by25(grid, grid_size):

    model = cp.CpModel()
    n = grid_size
    x = {}
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i * grid_size  + j] != "0":
                x[i, j] = str2Int[grid[i * grid_size + j]]
                # 这里规定：初始已经给了的数字不变
            else:
                x[i, j] = model.NewIntVar(1, grid_size, 'x[{},{}]'.format(i,j) ) 
                # 这里规定：余下的数字从1到9
    
    # ======= 下面是约束 ==========
    # 行和列必须不重复
    for i in range(n):
        row = [x[i, j] for j in range(n)]
        model.AddAllDifferent(row)
        col = [x[j, i] for j in range(n)]
        model.AddAllDifferent(col)

    # 25 * 25 的宫内数字不能重复
    for i in range(5):
        for j in range(5):
            cell = [
                x[r, c]
                for r in range(i * 5, i * 5 + 5)
                for c in range(j * 5, j * 5 + 5)
            ]
            model.AddAllDifferent(cell)

    solver = cp.CpSolver() 
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        for i in range(n):
            for j in range(n):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


if __name__ == "__main__":
    grid = "0G0B0UJ00H00000E00KVFR0PM0UEPSG0Q00O00000000W00I000000VXWTA00JCMR000PD0Y00B0WO00000ME0DK00X00J0AGNSQ0AFHL0B0DOEY00PI0CGN00000T000CP0OQ00000V0WRM0E0HI0E00UQ0GXWNY000000FB00T0K0000N000D00LQJ0UK0T0I0OR00VI00R00AEBMTOD0N000Q0S0J0K0XO0I0R0FS0000GV00UBN0MPS00TPNQ000J0LHOWX0Y0000D0DCG0000W0000SN000K0FQVE0000JLBH0000R0P0T0000OKWY0000MXYA0K000FB0000N0000SHC0N0000E0RLVWA0D000ICPX00JHT0MGV00PW0000AU0Y0L0JB0E0D0W0E000Y0UMKCQGI00N00TF00UJ0F0H0DT0EYB00P000Q0000F0Q00MG000000WBEJS0LA00D0PV0O0NSB0D00000CX0TM000H00000BSY0RX00LHCM0T0VEJQ0IYQRE0A00V00GT0PB00000DN0A00S0OF000CVRP00NDQEW000000H00D00000000M00A0KTFPR0CL0DNJP00G00000H00XR0B0A0"
    main_25by25(grid, 25)

24 7 25 2 4 21 10 14 3 8 1 12 9 23 19 5 20 17 11 22 6 18 15 16 13 
10 21 5 16 19 7 25 17 6 11 15 8 20 24 14 13 1 2 18 23 3 4 9 22 12 
17 11 14 9 22 24 23 20 1 19 7 10 3 13 18 6 12 15 16 4 8 25 21 5 2 
18 23 15 3 20 12 9 16 13 5 2 4 11 22 6 24 8 21 10 25 1 7 14 19 17 
13 1 6 8 12 18 2 22 4 15 5 25 17 21 16 9 19 3 7 14 10 11 23 24 20 
20 19 4 7 3 16 12 15 17 10 11 2 14 6 22 1 23 18 13 24 5 21 8 9 25 
5 13 1 21 17 3 7 24 23 14 25 16 8 18 9 10 15 6 2 19 4 20 12 11 22 
16 2 23 14 6 19 8 4 22 13 12 17 10 7 21 11 25 20 5 9 24 15 18 3 1 
22 9 12 25 18 11 21 1 5 2 13 20 15 4 24 14 16 8 3 17 7 19 6 10 23 
11 8 24 15 10 9 20 18 25 6 19 3 23 1 5 7 22 12 4 21 2 14 17 13 16 
19 22 18 20 16 14 17 6 7 3 10 11 12 8 15 23 24 5 25 2 9 13 1 4 21 
4 3 7 1 9 13 15 23 20 16 21 24 19 14 25 12 10 11 8 6 17 22 5 2 18 
21 5 10 12 2 8 22 9 19 24 18 13 16 3 20 4 17 7 1 15 11 23 25 6 14 
23 17 13 24 25 1 4 11 10 21 9 6 2 5 7 20 18 14 22 16 15 12 19 8 3 
15 14 11 6 8 25 5 2 18 12 22 23 1 17 4 19 21 13 9 3 16 24 20 7

![Alphabetic Sudoku](./assets/figures/Board.png)

# 解谜游戏：杀手数独（Killer Sudoku）


- 标准数独规则适用
- 除了标准数独规则外，限制每个虚线框内的数字之和等于标记的数字。


-------

English Version:

Killer Sudoku, Rules:

- Standard Sudoku Applied.
- Sum of numbers in cage equals the marked number.

In [2]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

def calc(model, cc, x, res):
    # 该函数让Model中增加一个约束：每个小宫内数字和为res，小宫内数字不重复
    cage = [x[i[0], i[1]] for i in cc]
    model.Add(sum(cage) == res)
    model.AddAllDifferent(cage)

def main():

    model = cp.CpModel()
    n = 9
    # 规定杀手数独的特殊表示方法： http://en.wikipedia.org/wiki/File:Killersudoku_color.svg
    # list( 小的宫内数字和, [ 宫内元素的坐标 ] )

    problem = [[3, [[0, 0], [0, 1]]],
        [15, [[0, 2], [0, 3], [0, 4]]],
        [22, [[0, 5], [1, 4], [1, 5], [2, 4]]],
        [4, [[0, 6], [1, 6]]],
        [16, [[0, 7], [1, 7]]],
        [15, [[0, 8], [1, 8], [2, 8], [3, 8]]],
        [25, [[1, 0], [1, 1], [2, 0], [2, 1]]],
        [17, [[1, 2], [1, 3]]],
        [9, [[2, 2], [2, 3], [3, 3]]],
        [8, [[2, 5], [3, 5], [4, 5]]],
        [20, [[2, 6], [2, 7], [3, 6]]],
        [6, [[3, 0], [4, 0]]],
        [14, [[3, 1], [3, 2]]],
        [17, [[3, 4], [4, 4], [5, 4]]],
        [17, [[3, 7], [4, 6], [4, 7]]],
        [13, [[4, 1], [4, 2], [5, 1]]],
        [20, [[4, 3], [5, 3], [6, 3]]],
        [12, [[4, 8], [5, 8]]],
        [27, [[5, 0], [6, 0], [7, 0], [8, 0]]],
        [6, [[5, 2], [6, 1], [6, 2]]],
        [20, [[5, 5], [6, 5], [6, 6]]],
        [6, [[5, 6], [5, 7]]],
        [10, [[6, 4], [7, 3], [7, 4], [8, 3]]],
        [14, [[6, 7], [6, 8], [7, 7], [7, 8]]],
        [8, [[7, 1], [8, 1]]],
        [16, [[7, 2], [8, 2]]],
        [15, [[7, 5], [7, 6]]],
        [13, [[8, 4], [8, 5], [8, 6]]],
        [17, [[8, 7], [8, 8]]]]

    x = {}
    for i in range(n):
        for j in range(n):
            x[i, j] = model.NewIntVar(1, n, "x[%i,%i]" % (i, j))

    x_flat = [x[i, j] for i in range(n) for j in range(n)]
    
    # ======= 下面是约束 ==========
    # 行和列必须不重复
    for i in range(n):
        row = [x[i, j] for j in range(n)]
        model.AddAllDifferent(row)
        
        col = [x[j, i] for j in range(n)]
        model.AddAllDifferent(col)

    # 3 * 3 的宫内数字不能重复
    for i in range(2):
        for j in range(2):
            cell = [
                x[r, c]
                for r in range(i * 3, i * 3 + 3)
                for c in range(j * 3, j * 3 + 3)
            ]
            model.AddAllDifferent(cell)

    # 小宫内的约束
    for (res, segment) in problem:
        calc(model,segment, x, res)
        
    solver = cp.CpSolver() 
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        for i in range(n):
            for j in range(n):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


if __name__ == "__main__":
    main()

2 1 5 6 4 7 3 9 8 
3 6 8 9 5 2 1 7 4 
7 9 4 3 8 1 6 5 2 
5 8 6 2 7 4 9 3 1 
1 4 2 5 9 3 8 6 7 
9 7 3 8 1 6 4 2 5 
8 2 1 7 3 9 5 4 6 
6 5 9 4 2 8 7 1 3 
4 3 7 1 6 5 2 8 9 

NumConflicts: 0
NumBranches: 0
WallTime: 0.013869000000000001


![Killer Sudoku](./assets/figures/KillerSudoku.png)

----------


## 解决加密难题

In [ ]:
"""Cryptarithmetic puzzle.

First attempt to solve equation CP + IS + FUN = TRUE
where each letter represents a unique digit.

This problem has 72 different solutions in base 10.
"""
from ortools.constraint_solver import pywrapcp


def main():
    # Constraint programming engine
    solver = pywrapcp.Solver("CP is fun!")

    base = 10

    # Decision variables.
    digits = list(range(0, base))
    digits_without_zero = list(range(1, base))
    c = solver.IntVar(digits_without_zero, "C")
    p = solver.IntVar(digits, "P")
    i = solver.IntVar(digits_without_zero, "I")
    s = solver.IntVar(digits, "S")
    f = solver.IntVar(digits_without_zero, "F")
    u = solver.IntVar(digits, "U")
    n = solver.IntVar(digits, "N")
    t = solver.IntVar(digits_without_zero, "T")
    r = solver.IntVar(digits, "R")
    e = solver.IntVar(digits, "E")

    # We need to group variables in a list to use the constraint AllDifferent.
    letters = [c, p, i, s, f, u, n, t, r, e]

    # Verify that we have enough digits.
    assert base >= len(letters)

    # Define constraints.
    solver.Add(solver.AllDifferent(letters))

    # CP + IS + FUN = TRUE
    solver.Add(
        p + s + n + base * (c + i + u) + base * base * f
        == e + base * u + base * base * r + base * base * base * t
    )

    solution_count = 0
    db = solver.Phase(letters, solver.INT_VAR_DEFAULT, solver.INT_VALUE_DEFAULT)
    solver.NewSearch(db)
    while solver.NextSolution():
        print(letters)
        # Is CP + IS + FUN = TRUE?
        assert (
            base * c.Value()
            + p.Value()
            + base * i.Value()
            + s.Value()
            + base * base * f.Value()
            + base * u.Value()
            + n.Value()
            == base * base * base * t.Value()
            + base * base * r.Value()
            + base * u.Value()
            + e.Value()
        )
        solution_count += 1
    solver.EndSearch()
    print(f"Number of solutions found: {solution_count}")


if __name__ == "__main__":
    main()

[C(2), P(3), I(7), S(4), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(3), I(7), S(5), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(5), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(6), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(8), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(8), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(3), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(8), F(9), U(6), N(3), T(1), R(0), E(5)]
[C(2), P(5), I(7), S(3), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(5), I(7), S(3), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(6), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(8), F(9), U(4), N(3), T(1), R(0), E(6)]
[C(2), P(6), I(7), S(3), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(6), I(7), S(5), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(8), I(7), S(3), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(8), I(7), S(3), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(8), I(7), S(4),

In [ ]:
# https://cn.gridpuzzle.com/jigsaw-killer-sudoku

------------

# 解决思维谜题：”帐篷”

规则和案例可以见：[在线链接](https://cn.puzzle-tents.com/?size=8)

帐篷的规则 很简单:

- 在与树水平或竖直方向相邻的格子里放置帐篷。帐篷与树一一对应。
- 帐篷之间互不相邻（即便是对角也不行）
- 边框外的数字表示每行、每列的帐篷总数。(最后的实例忘记截了...)

-----

输入数据：

1. 图的规模；
2. 树的位置；
3. 行、列外的数字

输出数据：

帐篷的位置

In [ ]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

def GetNeighbours(N, pos):
    
    neighbours = []
    directions = [[1,0], [-1,0], [0,1], [0,-1]]
    for direction in directions:
        if pos[0] + direction[0] >= N or pos[0] + direction[0] < 0 \
            or pos[1] + direction[1] >= N or pos[1] + direction[1] < 0:
                continue
        neighbours.append((pos[0] + direction[0], pos[1] + direction[1]))
    
    return neighbours

def TentSolver(N, trees, rows, cols):

    if len(rows) != N or len(cols) != N:
        raise Exception("N和输入数据长度不符")

    model = cp.CpModel()
    x = {}
    for i in range(N):
        for j in range(N):
            x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
    
    for tree in trees:
        neighbours = GetNeighbours(N, tree)
        cage = [x[i[0], i[1]] for i in neighbours]
        model.Add(sum(cage) >= 1)
        model.Add(x[tree[0], tree[1]] == 0)
    
    for idx, row in enumerate(rows):
        model.Add(sum([x[i, idx] for i in range(N)]) == row)
    
    for idx, col in enumerate(cols):
        model.Add(sum([x[idx, j] for j in range(N)]) == col)
    
    
    for i in range(N - 1):
        for j in range(N - 1):
            model.Add(sum([x[i, j], x[i + 1, j], x[i, j + 1], x[i + 1, j + 1]]) <= 1)
        

    solver = cp.CpSolver()
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(N):
            for j in range(N):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
        

if __name__ == "__main__":
    # rows = [3,0,2,0,2,0]
    # cols = [1,0,2,1,2,1]
    # trees = [
    #     (1,0),
    #     (2,1),
    #     (3,0),
    #     (3,3),
    #     (3,4),
    #     (4,5),
    #     (5,1)
    # ]
    # TentSolver(6, trees, rows, cols)
    
    rows = [5,2,4,1,4,1,3,2,4,4,3,3,3,3,3]
    cols = [6,1,3,3,2,2,5,3,4,1,2,3,3,1,6]
    trees = [
        (0,5), (0,6), (0,14),
        (1,0), (1,1),(1,2), (1,9), (1,10), (1,13),
        (2,8),
        (3,5),
        (4,0),(4,7),(4,8),(4,11),
        (5,2),(5,5),(5,8),(5,11),(5,14),
        (6,0),(6,11),
        (7,4),(7,5),(7,13),
        (8,5),(8,7),(8,11),
        (9,2),(9,9),(9,13),
        (10,0),
        (11,5),
        (12,1),(12,6),(12,10),(12,11),(12,13),
        (13,8),(13,12),
        (14,1),(14,3),(14,6),(14,7),(14,13)
    ]
    TentSolver(15, trees, rows, cols)

1 0 1 0 1 0 0 1 0 1 0 0 0 1 0 
0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 
0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 
0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 
1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 
0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 
0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 
1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 
1 0 1 0 0 1 0 0 1 0 0 0 1 0 1 

NumConflicts: 0
NumBranches: 262
WallTime: 0.055154



![](./assets/figures/TentExample.png)

![](./assets/figures/TentExample2.png)


--------

# 解决思维谜题：“马赛克”

马赛克 是一种类似扫雷的变体逻辑谜题.

- 你需要涂黑一些方格. 
- 数字表示包括它所在方格在内及周围一共九个方格内涂黑的方格数. 


> 注意这个和数独不大相同，因为可能存在数字为0的情况（九宫格没有涂黑的元素）。

In [ ]:
def GetNinePossibleNeighbours(X, Y, pos):
    
    neighbours = []
    directions = [-1, 0, 1]
    
    for ydir in directions:
        for xdir in directions:
            if (pos[0] + ydir >= Y or pos[0] + ydir < 0) or (pos[1] + xdir >= X or pos[1] + xdir < 0):
                continue
            neighbours.append((pos[0] + ydir, pos[1] + xdir))
    
    return neighbours

def MosaicSolver(X, Y, grid):
    print(len(grid))
    if len(grid) != X*Y:
        raise Exception("Grid 的网格和XY不符")

    model = cp.CpModel()
    x = {}
    for i in range(Y):
        for j in range(X):
            x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
    
    for idx, num in enumerate(grid):
        
        if num not in "0123456789*":
            raise Exception("检查输入格式, 只允许输入字符, 0123456789*")
        
        if num in "0123456789":
            x_idx , y_idx = idx % X, idx // Y
            neighbours = GetNinePossibleNeighbours(X, Y, (y_idx, x_idx))
            cage = [x[i[0], i[1]] for i in neighbours]
            model.Add(sum(cage) == int(num))
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()
        
        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
    
    else:
        print("Can't find Optimal.")
    
if __name__ == "__main__":
    iptStr = "222*3**33***5**4****34*****564447874224*2*245**5*3*36*6*245*3*46*****5*5**43456***58**56*****4*6*76*25**77*4**86***6655**5*8**455**54*44455***45**334**53*4**343244*334******54*455**3**22*24***3*4*4*****43**4*454**6*33*4*223********4*****5****357*6****4*666454*345*7****444**5*775**45*7**64*2355**565*1445464*****7***545*0****5*5***6***5*46**24**4*33**56*6*546****586543**434***56**3***532***1*332123*"
    
    MosaicSolver(20,20, iptStr)

    

400
1 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 
0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 
1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 0 0 0 1 1 
1 0 0 1 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 0 
1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 
0 0 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 
0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 0 1 
1 0 1 1 1 0 1 1 1 0 1 0 1 1 0 0 0 1 1 0 
0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 0 
0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0 1 
0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 
0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 0 0 0 1 0 
1 0 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 
0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 0 1 0 
0 1 1 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 0 
0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 
0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 
0 0 0 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 
0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 
1 1 0 1 1 1 1 0 1 0 1 0 0 0 1 0 0 0 1 0 

NumConflicts: 1
NumBranches: 190
WallTime: 0.018861000000000003


![](./assets/figures/Mosaic.png)

![](./assets/figures/Mosaic2.png)

# 解决思维谜题：连续数独（Consecutive Sudoku）

- 标准数独规则适用；
- 有标记的相邻的两个格子数字必须是连续的（差的绝对值是1，因此在实现的时候用 `AllAbsEquality` 函数）

------

English Version:

**Solve Consecutive Sudoku**

- Standard Sudoku Rules applied.
- Two adjacent cage with marker bust be integer-consecutive, (e.g. 1,2 or 2,3; 4,5 or 3,4)

In [ ]:
# https://github.com/CPMpy/cpmpy/

# nonogram 一款很有意思的游戏


In [33]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

def consecutive_sudoku_solver(grid, grid_size, consecutive_cage):
    
    model = cp.CpModel()
    n = grid_size 
    x = {}
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i * grid_size + j] != "0":
                x[i, j] = int(grid[i * grid_size + j])
            else:
                x[i, j] = model.NewIntVar(1, grid_size, name = f"x[{i}, {j}]")

    # for idx in range(len(consecutive_cage)):
    #     y[idx, 0] = model.NewIntVar(0,1, name = f"y-{idx}+")
    #     y[idx, 1] = model.NewIntVar(0,1, name = f"y-{idx}-")
    
    for i in range(n):
        row = [x[i, j] for j in range(n)]
        model.AddAllDifferent(row)
        col = [x[j, i] for j in range(n)]
        model.AddAllDifferent(col)
    
    for i in range(2):
        for j in range(2):
            cell = [
                x[r, c] 
                for r in range(i * 3, i * 3 + 3)
                for c in range(j * 3, j * 3 + 3)
            ]
            model.AddAllDifferent(cell)

    for _, cage in enumerate(consecutive_cage):
        model.AddAbsEquality(1, x[cage[0][0], cage[0][1]] - x[cage[1][0], cage[1][1]])
        # print(x[cage[0][0], cage[0][1]])
        # print(x[cage[1][0], cage[1][1]])
        

    solver = cp.CpSolver() 
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(n):
            for j in range(n):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())

    else:
        print("Unable to solve")
    
if __name__ == "__main__":
    grid = "600005000100000000000700000000000800000000000500100000000000000900000000000000000"
    consecutive_cage = [
        [(0, 8),  (1, 8) ],
        [(1, 6),  (1, 7) ],
        [(1, 6),  (2, 6) ],
        [(2, 6),  (2, 7) ],
        [(2, 3),  (3, 3) ],
        [(2, 4),  (3, 4) ],
        [(3, 0),  (4, 0) ],
        [(3, 3),  (4, 3) ],
        [(3, 7),  (4, 7) ],
        [(3, 8),  (4, 8) ],
        [(4, 2),  (4, 3) ],
        [(4, 4),  (5, 4) ],
        [(4, 6),  (4, 7) ],
        [(5, 0),  (5, 1) ],
        [(5, 1),  (6, 1) ],
        [(5, 2),  (5, 3) ],
        [(5, 3),  (6, 3) ],
        [(5, 4),  (6, 4) ],
        [(5, 7),  (6, 7) ],
        [(5, 8),  (6, 8) ],
        [(6, 1),  (6, 2) ],
        [(6, 3),  (7, 3) ],
        [(6, 6),  (6, 7) ],
        [(7, 0),  (7, 1) ],
        [(7, 1),  (7, 2) ],
        [(7, 3),  (7, 4) ],
        [(7, 4),  (8, 4) ],
        [(7, 7),  (8, 7) ],
        [(7, 8),  (8, 8) ],
        [(8, 7),  (8, 8) ],

    ]
    
    consecutive_sudoku_solver(grid, 9, consecutive_cage = consecutive_cage)


6 3 8 4 1 5 2 7 9 
1 7 5 9 6 2 4 3 8 
4 2 9 7 3 8 5 6 1 
7 9 3 6 2 4 8 1 5 
8 1 6 5 7 9 3 2 4 
5 4 2 1 8 3 6 9 7 
3 5 4 2 9 1 7 8 6 
9 8 7 3 4 6 1 5 2 
2 6 1 8 5 7 9 4 3 

NumConflicts: 0
NumBranches: 0
WallTime: 0.045585



![](./assets/figures/ConsecutiveSudoku.png)

![](./assets/figures/ConsecutiveSudoku1.png)


这个题竟然不是唯一解～～

https://github.com/CPMpy/cpmpy/

nonogram 一款很有意思的游戏

Silther Link Game，有翻译叫“数回”

> https://cs.carleton.edu/cs_comps/1516/slither/index.php


# 解谜游戏：Akari(AKA: Light Up!)

- 在空白格子里放置灯，灯可以照亮横纵所有行，除非遇到黑色格子的阻碍；
- 黑色格子里的数字表示这个黑色格子上下左右四个位置的灯的数量

输入数据：棋盘（用W表示白色格子，B表示黑色格子，数字表示“既是黑色格子又是数字”的格子，棋盘的长、宽；

输出数据：放置的情况

- 约束条件：
    - 数字格的灯约束必须满足；
    - 对每个垂直 / 水平的连续白色格形成的宫，里面的灯泡不能超过1（不然就会存在互相照亮这种情况）
    - 对每个空白格子，搜索上下左右四个方向以及自己的位置（直到在所有方向都到达边界或者遇到黑色格子），这么多格子里必须有至少一盏灯（这个约束限制了每个白色格子都必须被照亮）

-------


- Constraints:
    1. Lights around Number cell must be satisfied;
    2. Every Horizontal / vertical consecutive cell(s) must have no more than 1 light(Because light can't light up each other)
    3. Every white cell, search its horizontal / vertical direction(before stopped by the black wall or reaching margin) and must have at least one light

In [24]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp


def akari_solver(grid, X, Y):
    
    try:
        len(grid) == X * Y
    except Exception:
        raise(f"检查输入棋盘的长度 length of grid != {X}*{Y}")
    
    def akari_cross_neighbours(pos_x, pos_y):
    # Cross site
        res = []
        if pos_x + 1 < Y and grid[(pos_x + 1) * Y + pos_y] == "W":
            res.append(x[pos_x + 1, pos_y])
        
        if pos_x - 1 >= 0 and grid[(pos_x - 1) * Y + pos_y] == "W":
            res.append(x[pos_x - 1, pos_y])
        
        if pos_y + 1 < X and grid[pos_x * Y + pos_y + 1] == "W":
            res.append(x[pos_x, pos_y + 1])
        
        if pos_y - 1 >= 0 and grid[pos_x * Y + pos_y - 1] == "W":
            res.append(x[pos_x, pos_y - 1])

        
        return res
    
    def akari_neighbours(pos_x , pos_y):
        result = []
        ticks = 1
        while pos_y + ticks < X and grid[pos_x  * X + pos_y + ticks] == "W":
            result.append(x[pos_x, pos_y + ticks])
            ticks += 1
        ticks = 1
        while pos_y - ticks >= 0 and grid[pos_x  * X + pos_y - ticks] == "W":
            result.append(x[pos_x, pos_y - ticks])
            ticks += 1
        
        ticks = 1
        while pos_x - ticks >= 0 and grid[(pos_x - ticks) * X + pos_y] == "W":
            result.append(x[pos_x - ticks, pos_y])
            ticks += 1
        
        ticks = 1
        while pos_x + ticks < Y and grid[(pos_x + ticks) * X + pos_y] == "W":
            result.append(x[pos_x + ticks, pos_y])
            ticks += 1
        
        result.append(x[pos_x, pos_y])
        return result


    model = cp.CpModel()
    x = {}
    grid_str = ["" for i in range(Y)]
    
    for i in range(Y):
        for j in range(X):
            if grid[i * Y + j] == "W":
                x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
            else:
                x[i, j] = 0
            grid_str[i] += grid[i * Y + j]
            
    for i in range(Y):
        tmp_cabin = []
        for j in range(X):
            if grid_str[i][j] == "W":
                tmp_cabin.append(x[i, j])
                if j == X - 1:
                    model.Add(sum(tmp_cabin) <= 1)
            else:
                if len(tmp_cabin) > 0:
                    model.Add(sum(tmp_cabin) <= 1)
                tmp_cabin = []
                if grid_str[i][j] in "0123456789":
                    light_constr = akari_cross_neighbours(i, j)
                    model.Add(sum(light_constr) == int(grid_str[i][j]))
                
    for j in range(X):
        tmp_cabin = []
        for i in range(Y):
            if grid_str[i][j] == "W":
                tmp_cabin.append(x[i, j])
                if i == Y - 1:
                    model.Add(sum(tmp_cabin) <= 1)
            elif len(tmp_cabin) > 0:
                model.Add(sum(tmp_cabin) <= 1)
                tmp_cabin = []
            

    # Constraints: Lights in every small cage must <= 1
        # print(f"{pos_x}, {pos_y}")
        # print(result)
        # return result
        
    for idx, c in enumerate(grid):
        
        if c == "W":
            tobe_lighted = akari_neighbours(idx // X, idx % X)
            model.Add(sum(tobe_lighted) >= 1)
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
        
    

if __name__ == "__main__":
    
    grid = "".join([
        "BWWW2WWWWWWWBWWWB",
        "WWHWWWWHWHWWWW1WW",
        "W1WWW2WWWWW2WWWBW",
        "WWWBWWWWBWWWW2WWW",
        "1WWW2WWWWWWWBWWWB",
        "WW2WWWWBWBWWWW2WW",
        "W1WWBWBWWWBWBWWBW",
        "WWW1WWWWWWWWWBWWW",
        "0WWWWBWWWWW2WWWW0",
        "WWW1WWWWWWWWWBWWW",
        "W1WWBWBWWW1WBWW1W",
        "WWBWWWW2W1WWWW1WW",
        "1WWW2WWWWWWWBWWWB",
        "WWW1WWWW0WWWW3WWW",
        "W1WWW2WWWWW1WWW1W",
        "WW1WWWW1WBWWWWBWW",
        "BWWW1WWWWWWW1WWWB"
    ])
    # pass 
    akari_solver(grid , 17, 17)

0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 
1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 
0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 
0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 
0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 
0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 
0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 
1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 
0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 
0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 
1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 
0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 

NumConflicts: 0
NumBranches: 0
WallTime: 0.010498



![](./assets/figures/Atari.png)


![](./assets/figures/Atari1.png)


> 特别感谢Bilibili UP 的[这个视频](https://www.bilibili.com/video/BV1nE411s7cx/)
>
> 以及[这个实现链接](https://github.com/Borroot/akari)的帮助

---------------

# 解谜游戏：扫雷（Minesweeper）

- 规则和那个经典版扫雷有点子不同，这里只给一些数字，然后一些空格，负责在空格上插旗即可，不小心点到炸弹也不会爆炸，在作答完毕后统一结算。
- 数字表示周围八个格子里炸弹的数量。



In [10]:
from ortools.sat.python import cp_model as cp 

def minesweeper_solver(grid, X, Y):
    try:
        len(grid) == X*Y
    except Exception:
        raise(f"检查棋盘的长度： length of grid != {X}*{Y}")   

 
    model = cp.CpModel()
    x = {}
    
    def minesweeper_neighbours(pos_x, pos_y):
        direct = [-1, 0, 1]
        res = []
        for augx in direct:
            for augy in direct:
                if augx == 0 and augy == 0:
                    continue
                if pos_x + augx >= 0 and pos_x + augx < Y and pos_y + augy >= 0 and pos_y + augy < X:
                    res.append(x[pos_x + augx, pos_y + augy])
        return res
    
    for i in range(Y):
        for j in range(X):
            if grid[i * X + j] == "*":
                x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
            else:
                x[i, j] = 0
    
    for i in range(Y):
        for j in range(X):
            if grid[i * X + j] in "0123456789":
                cages = minesweeper_neighbours(i, j)
                model.Add(sum(cages) == int(grid[i * X + j]))
            
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())      

if __name__ == "__main__":
    
    
    grid = "".join([
        "1*34**01*1***21",
        "*******1**1***1", 
        "2*4*3***2*14***", 
        "1***3********2*", 
        "233***22*1*3***", 
        "*****41*2**1***", 
        "*3*****23*22*3*", 
        "2*2**2*******3*", 
        "*4*******113**2", 
        "***4*0*1*******", 
        "**5**1******4*1", 
        "4******2312*2**", 
        "***1**1***3**21", 
        "3****0*222***2*", 
        "*12***0*****1**"
    ])
    minesweeper_solver(grid, 15, 15)

    # for k in grid:
    #     print(len(k))

0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 
0 1 1 1 1 0 0 0 1 0 0 0 1 1 0 
0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 
0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 
0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 
1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 
1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 
0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 
1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 
0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 
0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 
0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 
1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 
0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 
0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 

NumConflicts: 0
NumBranches: 0
WallTime: 0.021615000000000002



![](./assets/figures/Minesweeper.png)

![](./assets/figures/Minesweeper1.png)

------------


# 解谜游戏：NoriNori（海苔？）


你需要按如下方式为一些方格涂黑:
- 一个区域内有且仅有两个方格被涂黑.
- 每个涂黑的方格都是一个多米诺*的一部分. 多米诺可以跨过区域边界.
- 多米诺之间不能相连,但是对角线相接是可以的.

> 一个多米诺是相邻的两个方格组成的图形 (1x2 or 2x1).



In [15]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp 

def norinori_solver(grid_ipt, X, Y):
    grid = grid_ipt.split(",")
    try:
        len(grid ) == X * Y
    except Exception:
        raise(f"检查输入棋盘的长度 length of grid != {X} * {Y}")
    
    model = cp.CpModel()
    cages = {}
    x = {}
    for i in range(Y):
        for j in range(X):
            x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
    
    for idx, letter in enumerate(grid):
        if letter not in cages:
            cages[letter] = [x[idx // X, idx % X]]
        else:
            cages[letter].append(x[idx // X, idx % X])

    def norinori_neightbours(pos_x, pos_y):
        
        res = []
        
        if pos_x + 1 < Y:
            res.append(x[pos_x + 1, pos_y])
        
        if pos_x - 1 >= 0:
            res.append(x[pos_x - 1, pos_y])
        
        if pos_y + 1 < X:
            res.append(x[pos_x, pos_y + 1])
        
        if pos_y - 1 >= 0:
            res.append(x[pos_x, pos_y - 1])
        
        return res
    
    for i in range(Y):
        for j in range(X):
            neighbours = norinori_neightbours(i, j)
            model.Add(sum(neighbours) == 1).OnlyEnforceIf(x[i, j])
    
    for _, value in cages.items():
        model.Add(sum(value) == 2)
    
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


# 规定一下输入的grid的形状

if __name__ == "__main__":
    grid = "1,1,1,2,2,2,3,3,1,1,7,7,3,4,4,1,7,8,5,5,4,4,4,8,4,4,4,6,4,8,4,6,6,6,6,6"
    norinori_solver(grid, 6, 6)
    
    # 这里用逗号隔开是因为真的有可能出现很多很多很多很多宫啊！
 

0 0 1 1 0 1 
1 1 0 0 0 1 
0 0 0 1 1 0 
1 1 0 0 0 1 
0 0 1 1 0 1 
1 1 0 0 0 0 

NumConflicts: 0
NumBranches: 0
WallTime: 0.0032140000000000003



![](./assets/figures/Norinori1.png)

![](./assets/figures/Norinori.png)

-------

# 解谜游戏：Number Link（数链？）

- 用一条连续的线把相同的数字连接起来；
- 线只能从上下左右四个方向穿过图案，不能两次经过同一个格子；
- 线不能分支、不能交叉、不能经过数字格；


----------------

- Connect pairs of the same numbers with a continuous line.
- Lines go through the center of the cells, horizontally, vertically, or changing direction, and never twice through the same cell.
- Lines cannot cross, branch off, or go through the cells with numbers.

In [7]:

from ortools.sat.python import cp_model as cp

def numberlink_solver(grid_ipt , X, Y):
    
    grid = grid_ipt.split(",")
    try:
        len(grid) == X * Y
    except Exception:
        raise(f"检查输入棋盘的长度： length of grid != {X} * {Y}")

    ods = max(map(int, grid))
    model = cp.CpModel()
    x = {}
    y = {} 
    # 表示每个格子属于第几个线路
    
    def numberlink_neighbours(pos_x, pos_y, k):
        res = []
        if pos_x + 1 < Y:
            res.append(x[pos_x + 1, pos_y, k])
                
        if pos_x - 1 >= 0:
            res.append(x[pos_x - 1, pos_y, k])
        
        if pos_y + 1 < X :
            res.append(x[pos_x, pos_y + 1, k])
        
        if pos_y - 1 >= 0 :
            res.append(x[pos_x, pos_y - 1, k]) 
                
        return res
    

    for i in range(Y):
        for j in range(X):
            if grid[i * X + j] == "0":
                y[i, j] = model.NewIntVar(1, ods, name = f"y[{i}, {j}]")
            else:
                y[i, j] = int(grid[i * X + j])
            
            if i + 1 < Y:
                x[i, j , 1, 0] = model.NewBoolVar(name = f"x[{i}, {j}, 1, 0]")
            else:
                x[i, j , 1, 0] = 0
            
            if i - 1 >= 0:
                x[i, j , -1, 0] = model.NewBoolVar(name = f"x[{i}, {j}, -1, 0]")
            else:
                x[i, j , -1, 0] = 0
                
            if j + 1 < X:
                x[i, j , 0, 1] = model.NewBoolVar(name = f"x[{i}, {j}, 0, 1]")
            else:
                x[i, j , 0, 1] = 0
            
            if j - 1 >= 0:
                x[i, j , 0, -1] = model.NewBoolVar(name = f"x[{i}, {j}, 0, -1]")
            else:
                x[i, j , 0, -1] = 0
            
            if grid[i * X + j] != "0":
                model.Add(x[i, j , 1, 0] + x[i, j , -1, 0] + x[i, j ,0 , -1] + x[i, j , 0, 1] == 1)
            else:
                model.Add(x[i, j , 1, 0] + x[i, j , -1, 0] + x[i, j ,0 , -1] + x[i, j , 0, 1] == 2)
    
    for i in range(Y):
        for j in range(X):
            if j - 1 >= 0:
                model.Add(y[i, j] == y[i, j - 1]).OnlyEnforceIf(x[i, j , 0, -1])
            if j + 1 < X:
                model.Add(y[i, j] == y[i, j + 1]).OnlyEnforceIf(x[i, j , 0,  1])
            if i + 1 < Y:
                model.Add(y[i, j] == y[i + 1, j]).OnlyEnforceIf(x[i, j , 1, 0])
            if i - 1 >= 0:
                model.Add(y[i, j] == y[i - 1, j]).OnlyEnforceIf(x[i, j , -1, 0])

    
    solver = cp.CpSolver()
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(y[i, j]), end = " ")
            print()
        print()
        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())

    else:
        print("Unable to find the OPTIMAL.")

if __name__ == "__main__":
    grid = "0,11,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,2,0,0,0,0,9,0,0,1,0,11,0,0,3,0,10,9,0,3,0,0,0,0,0,0,0,0,0,0,0,7,0,0,7,0,0,0,0,0,0,0,0,0,5,0,0,8,6,0,0,0,6,0,0,0,8,1,0,0,4,0,0,0,0,0,0,0,0,5"
    numberlink_solver(grid, 10, 10)

11 11 4 1 1 1 1 1 1 1 
11 2 4 1 3 3 3 3 3 1 
11 2 4 1 3 10 10 10 3 1 
11 2 4 1 3 10 9 9 3 1 
11 11 4 1 3 10 10 9 3 3 
4 4 4 1 1 1 1 1 1 1 
4 7 7 7 7 6 6 6 6 1 
4 5 5 5 5 6 8 8 6 1 
4 5 6 6 6 6 8 1 1 1 
4 5 5 5 5 5 5 5 5 5 

NumConflicts: 0
NumBranches: 664
WallTime: 0.059899



![](./assets/figures/NumberLink.png)

![](./assets/figures/NumberLink1.png)

In [26]:
# https://github.com/uguryavuz/numberlink-solver

5
